# Assignment Problem solved with Linear Programming
So a friend of mine is working at a university. Once a semester his students get the task to read and present a research paper. He is offering them a catalogue of suitable papers and they rate their favorite ones from prio 1 to 5.
After all students gave him their feedback, he is left with a table which looks something like this:  
![table_paper_students](table_paper_students.png)
The idea is that every paper is only selected by one student. His task now is to distrubute the papers to the students so that most of them get one of their top priorities. How to find the optimal distribution?

The titel does already spoil it, we can apply linear programming. We want to find a solution which minimizes the sum of all selected priorities. The priority can also be considered as a weight $w_{ij}$ of an edge between student $i$ and paper $j$ in an [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph#/media/File:Simple-bipartite-graph.svg).  
We need an additional variable $x_{ij}$ to describe the linear problem:  
Eq0. 
$$x_{ij} = \begin{cases} 1 \text{, if paper $j$ is assigned to student $i$}\\ 0 \text{, else} \end{cases}$$  
With these to variables the objective can be written as 

Eq1.  
$$\text{min } \sum_{(i,j \in AxT)} w_{ij} x_{ij}$$

The next step is to think about the constraints of the problem. Every student (out of $A$ total students) has to present one paper (out of $T$ total papers) which means  

Eq2. 
$$\sum_{j=1}^{T}{x_{ij}=1} \text{, for every student $i$}$$  

Another constrains is that each paper is only presented by one student, but if there are more papers then students then not all papers will be selected. We can describe these sentences as follows:  

Eq3. 
$$\sum_{i=1}^{A}{x_{ij} \leq 1} \text{, for every paper $j$}$$  



Sources:  
[Wikipedia - Assignment Problem](https://en.wikipedia.org/wiki/Assignment_problem)



### Example 
In this section we apply linear programming to a simplified problem which is described by the following code. Note that the students here only selected with prio 1-3. Since we want to minimize the problem we punish empty cells / not selected papers with a weight of 9 (other weights greater that 3 are feasible).

In [1]:
column_labels = ["Student A", "Student_B", "Student_C"]
row_labels    = ["Paper 1", "Paper 2", "Paper 3", "Paper 4","Paper 5"]
prio_matrix   = [[9, 9, 9], 
                 [1, 2, 2],
                 [9, 3, 9],
                 [2, 1, 1],
                 [3, 9, 3],]

columns = len(prio_matrix[0])
rows = len(prio_matrix)

We will use [lineprog](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html) of the [SciPy](https://docs.scipy.org/doc/scipy/reference/index.html) library to solve the optimization problem.
Since lineprog only solves linear equation, the function only expects $w$ as an input and will automatically minimize $w^T x$ internally.  
$$
w^T x = 
\begin{bmatrix} w_{11} & w_{12} & w_{13} & w_{21} & \dots & w_{51} & w_{52} & w_{53}\end{bmatrix}
\begin{bmatrix} x_{11} \\ x_{12} \\ x_{13} \\ x_{21} \\ \dots \\ x_{51} \\ x_{52} \\ x_{53} \end{bmatrix}
$$

In [2]:
w = [item for sublist in prio_matrix for item in sublist]  # [row1, row2, ...]

Next we need to formulate the equality constraints from Eq2. in the form:  
$A_{eq} x = b_{eq}$  
for our example:  
$$
\begin{bmatrix} 
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 
\end{bmatrix}
\begin{bmatrix} 
x_{11} \\ x_{12} \\ x_{13} \\ x_{21} \\ \dots \\ x_{51} \\ x_{52} \\ x_{53} 
\end{bmatrix}
=
\begin{bmatrix} 
1 \\ 1 \\ 1 
\end{bmatrix}
$$

In [3]:
A_eq = [[0 for i in range(columns*rows)] for j in range(columns)]
for m in range(columns):        
    for n in range(m, columns*rows, columns):
        A_eq[m][n] = 1
b_eq = [1]*columns

And the upper bound constraints from Eq3. need the form   
$A_{ub} x \leq b_{ub}$    
for our example:  
$$
\begin{bmatrix} 
1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
\end{bmatrix}
\begin{bmatrix} 
x_{11} \\ x_{12} \\ x_{13} \\ x_{21} \\ \dots \\ x_{51} \\ x_{52} \\ x_{53} 
\end{bmatrix}
\leq
\begin{bmatrix} 
1 \\ 1 \\ 1 \\ 1 \\ 1
\end{bmatrix}
$$

In [4]:
A_ub = [[0 for i in range(columns*rows)] for j in range(rows)]
for m in range(0,rows):
    for n in range(0, columns*rows):
        if m*columns <= n < m*columns+columns:
            A_ub[m][n] = 1
b_ub = [1]*rows  

Last we need the bounds of each $x_{ij}$ which is given by the definition in Eq.0

In [5]:
x_bounds = [(0, 1)]*columns*rows

Now we are ready to run the optimization

In [6]:
from scipy.optimize import linprog

linprog_result = linprog(c=w, 
                         A_ub=A_ub, b_ub=b_ub, 
                         A_eq=A_eq, b_eq=b_eq, 
                         bounds=x_bounds, method='interior-point')

# Map parameter result to a dict
x_result = linprog_result.x.tolist()
x_result = [int(round(el)) for el in x_result]
i = 0
dict = {} 
for row in row_labels:
     for col in column_labels:
        if  x_result[i] == 1:
            dict.update({col:row})
        i +=1
    
print(f"Cost: {int(linprog_result.fun)}")
print(dict)

Cost: 5
{'Student A': 'Paper 2', 'Student_B': 'Paper 3', 'Student_C': 'Paper 4'}


This is the solution for our example. Whats left to do is pack all the code shown above into a function so my friend is ready to throw his paper distributions at it each semester.

In [7]:
from scipy.optimize import linprog

def solve_assignment(cost_matrix, col_labels, row_labels):
    # Calculate matrix dimensions
    columns = len(cost_matrix[0])
    rows = len(cost_matrix)

    # Formulate optimization problem
    # Objective function is to minimize w*x
    w = [item for sublist in cost_matrix for item in sublist]  # [row1, row2, ...]
    
    # Inequality constraints
    # A_ub*x <= b
    A_ub = [[0 for i in range(columns*rows)] for j in range(rows)]
    for m in range(0,rows):
        for n in range(0, columns*rows):
            if m*columns <= n < m*columns+columns:
                A_ub[m][n] = 1
    b_ub = [1]*rows           
    # Equality constraints
    # A_eq*x = b
    A_eq = [[0 for i in range(columns*rows)] for j in range(columns)]
    for m in range(columns):        
        for n in range(m, columns*rows, columns):
            A_eq[m][n] = 1
    b_eq = [1]*columns
    
    # X bounds
    x_bounds = [(0, 1)]*columns*rows
    
    # Run optimization
    linprog_result = linprog(c=w, 
                             A_ub=A_ub, b_ub=b_ub, 
                             A_eq=A_eq, b_eq=b_eq, 
                             bounds=x_bounds, method='interior-point')
    # Beautify result
    # Get resulting parameters
    x_result = linprog_result.x.tolist()
    x_result = [int(round(el)) for el in x_result]
    # Map parameter result to labels
    i = 0
    dict = {} 
    for row in row_labels:
         for col in col_labels:
            if  x_result[i] == 1:
                dict.update({col:row})
            i +=1
    print(f"Cost: {int(linprog_result.fun)}")
    return dict

In [8]:
solve_assignment(prio_matrix, column_labels, row_labels)  

Cost: 5


{'Student A': 'Paper 2', 'Student_B': 'Paper 3', 'Student_C': 'Paper 4'}